<a href="https://colab.research.google.com/github/kobi3028/AttacksonImplementationsCourseBook/blob/master/Labs/CacheSizeTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Open the file in google colab https://colab.research.google.com/github/Yossioren/AttacksonImplementationsCourseBook/blob/master/Labs/CacheSizeTest.ipynb
import numpy as np
from scipy import stats

!pip install -q bokeh
from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.io import output_notebook
from bokeh.models.tickers import FixedTicker
from bokeh.models.formatters import PrintfTickFormatter
# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

import mmap
import time
import gc

import ctypes
!pip install -q keystone-engine
from keystone import *

ks = Ks(KS_ARCH_X86, KS_MODE_64)

     |████████████████████████████████| 1.8MB 5.8MB/s 


In [2]:
# rdtsc func 
rdtsc_asm = '''
rdtsc
shl rdx, 32
or  rax, rdx
ret
'''
byte_code, count = ks.asm(rdtsc_asm)
assert count == 5

rdtsc_buf = mmap.mmap(-1, mmap.PAGESIZE, prot=mmap.PROT_READ | mmap.PROT_WRITE | mmap.PROT_EXEC)
rdtsc_buf.write(bytes(byte_code))

fpointer = ctypes.c_void_p.from_buffer(rdtsc_buf)
func_type = ctypes.CFUNCTYPE(ctypes.c_uint64)
rdtsc = ctypes.cast(ctypes.addressof(fpointer), func_type)

In [3]:
#LENGTH = 256*1024*1024 #256MB
MAP_HUGETLB = 0x40000
MAP_POPULATE = 0x08000

NUM_READ = (1<<20)    # number of random array accesses  
NUM_TEST = 0x10       # test every array size NUM_TEST times
ARRAY_SIZE_POWER = 24 # size of the array for random access (2^(10+ARRAY_SIZE_POWER)) 

res_arr = {}

for i in range(ARRAY_SIZE_POWER):
  length = (1<<(i+10))
  res_arr[length] = []
  counter = 0
  
  #Create random sequence
  rand_arr = np.random.randint(low=0, high=256, size=1<<10)
  rand_arr = rand_arr.tobytes('C')[::8]
  #print(rand_arr[:2], rand_arr.tobytes('C')[::8][:2])
  
  #Allocate memory
  mm = mmap.mmap(-1, length=length, 
            flags=(mmap.MAP_ANONYMOUS|mmap.MAP_SHARED|MAP_POPULATE), 
            prot=(mmap.ACCESS_READ|mmap.ACCESS_WRITE))

  #Write random sequence
  for j in range(1<<i):
    mm.write(rand_arr)
    
  time.sleep(1)
  while counter < NUM_TEST:

    #Create random indexs array
    index_arr = np.random.randint(low=0, high=length, size=NUM_READ)
    
    index = 0
    #Memory access time measurement
    #start = time.time_ns()
    start = rdtsc()
    while index < index_arr.size:
      tmp = mm[index_arr[index]]
      index += 1
    #end = time.time_ns() - start  
    end = rdtsc() - start  
    
    #Save the average time
    res_arr[length].append(float(end)/NUM_READ)
    counter += 1
  
  #Clean memory
  mm.close()
  gc.collect()

#Print the results
for key in res_arr:
  print('{0:>#12x}'.format(key), res_arr[key])

       0x400 [1102.9469709396362, 973.6717739105225, 995.2278594970703, 977.9930801391602, 996.415153503418, 971.8231105804443, 984.123477935791, 974.4236555099487, 975.7940378189087, 977.5233449935913, 958.0894174575806, 1006.2560796737671, 959.6688585281372, 999.7633008956909, 997.1655836105347, 990.6295948028564]
       0x800 [1016.7450485229492, 999.1093683242798, 990.2256288528442, 979.692624092102, 974.6443643569946, 985.5371885299683, 979.4079418182373, 973.721848487854, 1001.6361103057861, 955.2102575302124, 980.4498443603516, 950.815408706665, 974.9831800460815, 979.2134132385254, 976.0586557388306, 985.6290349960327]
      0x1000 [945.797963142395, 1009.2634410858154, 957.1429672241211, 1007.8760623931885, 951.4429626464844, 994.8447065353394, 974.5471544265747, 962.9395666122437, 975.4107418060303, 1034.8406972885132, 1009.1173982620239, 983.5002918243408, 1013.0937604904175, 965.2290372848511, 991.3724851608276, 957.0136651992798]
      0x2000 [983.52308177948, 944.05195903

In [4]:
#Plot the results
p = figure(title='Memory Random Access Time Measurement', 
           x_axis_label='Array Size (KBytes)', 
           #y_axis_label='Random Access time [NS]', 
           y_axis_label='CPU tickes', 
           x_axis_type="log")

keys = [(key>>10) for key in res_arr.keys()]
p.xaxis.ticker = FixedTicker(ticks=keys)
p.xaxis.major_label_orientation = "vertical"
p.xaxis.formatter=PrintfTickFormatter(format="0x%X")
avg = []
for key in res_arr:
  #Plot the results
  p.scatter([key>>10]*len(res_arr[key]), res_arr[key])
  
  #Remove the outlier
  data = np.asarray(res_arr[key])
  data = data[abs(data - np.mean(data)) < 2 * np.std(data)]
  avg.append(np.mean(data))

#Plot the avg line
p.line(keys, avg, line_width=2, line_color='orange')
show(p)
